In [32]:
import sys

import pandas as pd
import numpy as np
#import matplotlib as plt
#from matplotlib import pyplot
#import scipy as sp
#from xgboost import XGBClassifier
import sklearn
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn import svm



from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import mean_squared_error, r2_score,accuracy_score,recall_score,precision_score, f1_score

from sklearn import preprocessing

from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV

from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit
import matplotlib.pyplot as plt



import joblib

In [2]:
filename = 'events.csv'
events = pd.read_csv('./data/' + filename)
shots = events[(events.event_type==1)]
shots_on_target = shots[(shots.shot_outcome==1)].copy()
shots_on_target['is_save'] = 1 - shots_on_target['is_goal']
shots_on_target.head()

,id_odsp,id_event,sort_order,time,text,event_type,event_type2,side,event_team,opponent,...,player_out,shot_place,shot_outcome,is_goal,location,bodypart,assist_method,situation,fast_break,is_save
13,UFot0hit/,UFot0hit14,14,17,"Goal! Borussia Dortmund 1, Hamburg 0. Kevin G...",1,12.0,1,Borussia Dortmund,Hamburg SV,...,NaN,4.0,1.0,1,9.0,2.0,1,1.0,0,0
22,UFot0hit/,UFot0hit23,23,26,Attempt saved. Mats Hummels (Borussia Dortmund...,1,12.0,1,Borussia Dortmund,Hamburg SV,...,NaN,5.0,1.0,0,3.0,3.0,2,1.0,0,1
26,UFot0hit/,UFot0hit27,27,29,"Goal! Borussia Dortmund 2, Hamburg 0. Mario G...",1,12.0,1,Borussia Dortmund,Hamburg SV,...,NaN,5.0,1.0,1,3.0,1.0,1,1.0,0,0
49,UFot0hit/,UFot0hit50,50,48,Attempt saved. Kevin Grosskreutz (Borussia Dor...,1,12.0,1,Borussia Dortmund,Hamburg SV,...,NaN,11.0,1.0,0,3.0,1.0,1,1.0,0,1
50,UFot0hit/,UFot0hit51,51,49,"Goal! Borussia Dortmund 3, Hamburg 0. Kevin G...",1,NaN,1,Borussia Dortmund,Hamburg SV,...,NaN,4.0,1.0,1,13.0,1.0,0,3.0,0,0


In [3]:
shots_prediction = shots_on_target.iloc[:,[-1,-6,-5,-4,-3, -2]]
dummies = pd.get_dummies(shots_prediction, columns=['location', 'bodypart','assist_method', 'situation'])
dummies.columns = ['is_save', 'fast_break','loc_centre_box', 'loc_diff_angle_lr', 'diff_angle_left', 'diff_angle_right', 'left_side_box', 'left_side_6ybox', 'right_side_box', 'right_side_6ybox', 'close_range', 'penalty', 'outside_box', 'long_range', 'more_35y', 'more_40y', 'not_recorded', 'right_foot', 'left_foot', 'header', 'no_assist', 'assist_pass', 'assist_cross', 'assist_header', 'assist_through_ball', 'open_play', 'set_piece', 'corner', 'free_kick']
dummies.head()

,is_save,fast_break,loc_centre_box,loc_diff_angle_lr,diff_angle_left,diff_angle_right,left_side_box,left_side_6ybox,right_side_box,right_side_6ybox,...,header,no_assist,assist_pass,assist_cross,assist_header,assist_through_ball,open_play,set_piece,corner,free_kick
13,0,0,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,1,0,0,0
22,1,0,1,0,0,0,0,0,0,0,...,1,0,0,1,0,0,1,0,0,0
26,0,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
49,1,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
50,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0


In [4]:
X = dummies.iloc[:,1:]
y = dummies.iloc[:,0]
print(X.shape)
print(y.shape)

(78014, 28)
(78014,)


In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.35, random_state=1)
clf = RandomForestClassifier(n_estimators=200) #81.63%
#clf = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(30,20,3), random_state=1 , max_iter = 1000) #81.52%
#clf = KNeighborsClassifier(n_neighbors=91, algorithm='auto',weights='uniform'); #80.23%
#clf = RandomForestClassifier(n_estimators=400) #80.23%
#clf = svm.SVC(kernel='linear') #81.54%
clf.fit(X_test, y_test)
accuracy = clf.score(X_test, y_test)
print('accuracy is {:.2f} %'.format(accuracy*100))

accuracy is 81.63 %


In [7]:
accuracy = clf.score(X_test, y_test)
y_pred = clf.predict_proba(X_test)
predict = clf.predict(X_test)
y_total = y_train.count()
y_positive = y_train.sum()
auc_roc = roc_auc_score(y_test, y_pred[:, 1])
print('The training set contains {} examples (shots on target) of which {} are positives (goals).'.format(y_total, y_positive))
print('The accuracy of classifying whether a shot is goal or not is {:.2f} %'.format(accuracy*100))
print('Our classifier obtains an AUC-ROC of {:.4f}.'.format(auc_roc))

The training set contains 50709 examples (shots on target) of which 35195 are positives (goals).
The accuracy of classifying whether a shot is goal or not is 81.63 %
Our classifier obtains an AUC-ROC of 0.8392.


In [ ]:
print('Confusion Matrix:')
print(confusion_matrix(y_test,predict))
print('Report:')
print(classification_report(y_test,predict))

In [ ]:
predictions = X_test.copy()
predictions['true_saves'] = y_test
predictions['expected_saves'] = y_pred[:,1]
predictions['difference'] = predictions['expected_saves'] - predictions['true_saves']
predictions = predictions.iloc[:,28:31]

In [ ]:
ypred2 = clf.predict_proba(X_train)
predictions_train = X_train.copy()
predictions_train['true_saves'] = y_train
predictions_train['expected_saves'] = ypred2[:,1]
predictions_train['difference'] = predictions_train['expected_saves'] - predictions_train['true_saves']
predictions_train = predictions_train.iloc[:,28:31]
all_predictions = pd.concat([predictions, predictions_train], axis=0)
events2 = pd.concat([events, all_predictions], axis=1)
shots2 = events2[events2.event_type==1]
shots2.head()

In [ ]:
xG_Teams = shots2[['opponent', 'event_type', 'true_saves', 'expected_saves', 'difference']].groupby('opponent').sum()
xG_Teams.columns = ['n_shots', 'saves_made', 'expected_saves', 'difference']
xG_Teams[['saves_made', 'expected_saves']].corr()

In [ ]:
xG_Teams.sort_values(['difference', 'saves_made'])

In [ ]:
xG_Teams.sort_values(['expected_saves'], ascending=False)

In [ ]:
xG_Teams['xG_per_shot_ratio'] = xG_Teams['expected_saves'] / xG_Teams['n_shots']
xG_Teams[xG_Teams.n_shots>1].sort_values(['xG_per_shot_ratio', 'saves_made'])

In [ ]:
outside_area = events2[(events2.event_type==1) & (events2.location==15)]
oA_Teams = outside_area[['opponent', 'event_type', 'true_saves', 'expected_saves', 'difference']].groupby('opponent').sum()
oA_Teams.columns = ['n_shots', 'saves_made', 'expected_saves', 'difference']
oA_Teams.sort_values(['expected_saves']) #pode meter se difference, etc

In [ ]:
### 3.1 Predict the winning team
An important characteristic to evaluate a team is its defense capabilities. In order to evaluate a team's defense, we can, as we do in this problem, predict the result of a shot on target. This way, we are able to find any vulnerabilities that a team's set of goalkeepers may have (location of the striker who shot the ball, its body part that made the shot, location of the goal where the ball was headed, etc), and compare several teams defensive power and liabilities.
#### Approach

In [ ]:
For this problem the dataset consisted of the games played over the course of 5 years, between 2011/2012 until 2016/2017, of the five major european leagues, suming to a total of 10112 games. We consider all of the shots on target of a game (event_type = 1, shot_outcome = 1) for this problem, the opponent team (in order to know the defensive team), so that we are able to predict if the keeper saves the shot.

In [8]:
shots = events[(events.event_type==1)]
shots_on_target = shots[(shots.shot_outcome==1)].copy()
shots_on_target['is_save'] = 1 - shots_on_target['is_goal']
shots_on_target.head()

,id_odsp,id_event,sort_order,time,text,event_type,event_type2,side,event_team,opponent,...,player_out,shot_place,shot_outcome,is_goal,location,bodypart,assist_method,situation,fast_break,is_save
13,UFot0hit/,UFot0hit14,14,17,"Goal! Borussia Dortmund 1, Hamburg 0. Kevin G...",1,12.0,1,Borussia Dortmund,Hamburg SV,...,NaN,4.0,1.0,1,9.0,2.0,1,1.0,0,0
22,UFot0hit/,UFot0hit23,23,26,Attempt saved. Mats Hummels (Borussia Dortmund...,1,12.0,1,Borussia Dortmund,Hamburg SV,...,NaN,5.0,1.0,0,3.0,3.0,2,1.0,0,1
26,UFot0hit/,UFot0hit27,27,29,"Goal! Borussia Dortmund 2, Hamburg 0. Mario G...",1,12.0,1,Borussia Dortmund,Hamburg SV,...,NaN,5.0,1.0,1,3.0,1.0,1,1.0,0,0
49,UFot0hit/,UFot0hit50,50,48,Attempt saved. Kevin Grosskreutz (Borussia Dor...,1,12.0,1,Borussia Dortmund,Hamburg SV,...,NaN,11.0,1.0,0,3.0,1.0,1,1.0,0,1
50,UFot0hit/,UFot0hit51,51,49,"Goal! Borussia Dortmund 3, Hamburg 0. Kevin G...",1,NaN,1,Borussia Dortmund,Hamburg SV,...,NaN,4.0,1.0,1,13.0,1.0,0,3.0,0,0


In [ ]:
For an initial approach, we select all of the shots that are made on goal. We add a new column "is_save" that is the opposite of the column "is_goal". We add this column so that we can ease the analysis and process of the data.

In [9]:
shots_prediction = shots_on_target.iloc[:,[-1,-6,-5,-4,-3, -2]]
dummies = pd.get_dummies(shots_prediction, columns=['location', 'bodypart','assist_method', 'situation'])
dummies.columns = ['is_save', 'fast_break','loc_centre_box', 'loc_diff_angle_lr', 'diff_angle_left', 'diff_angle_right', 'left_side_box', 'left_side_6ybox', 'right_side_box', 'right_side_6ybox', 'close_range', 'penalty', 'outside_box', 'long_range', 'more_35y', 'more_40y', 'not_recorded', 'right_foot', 'left_foot', 'header', 'no_assist', 'assist_pass', 'assist_cross', 'assist_header', 'assist_through_ball', 'open_play', 'set_piece', 'corner', 'free_kick']
dummies.head()

,is_save,fast_break,loc_centre_box,loc_diff_angle_lr,diff_angle_left,diff_angle_right,left_side_box,left_side_6ybox,right_side_box,right_side_6ybox,...,header,no_assist,assist_pass,assist_cross,assist_header,assist_through_ball,open_play,set_piece,corner,free_kick
13,0,0,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,1,0,0,0
22,1,0,1,0,0,0,0,0,0,0,...,1,0,0,1,0,0,1,0,0,0
26,0,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
49,1,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
50,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0


In [ ]:
We get 28 columns as part of the input and we get 1, "is_save" for the output.
Next we will split the data into the training one and the test one, so we can train our model.

In [13]:
X = dummies.iloc[:,1:]
y = dummies.iloc[:,0]
print(X.shape)
print(y.shape)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.35, random_state=1)
X_train

(78014, 28)
(78014,)


,fast_break,loc_centre_box,loc_diff_angle_lr,diff_angle_left,diff_angle_right,left_side_box,left_side_6ybox,right_side_box,right_side_6ybox,close_range,...,header,no_assist,assist_pass,assist_cross,assist_header,assist_through_ball,open_play,set_piece,corner,free_kick
219207,0,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
907777,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
803186,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,0
679015,0,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
887350,0,0,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265565,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
885851,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
607503,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
64781,0,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0


In [23]:
def checkClf(name,clf):
    clf.fit(X_train,y_train)
    y_pred = clf.predict_proba(X_test)
    print("Model name: " + name)
    #print("R2 score: {:.4f}".format(r2_score(y_test, y_pred)))
    #print("Mean square error: {:.4f}".format(mean_squared_error(y_test, y_pred)))
    #print('Precision: {:.4f}'.format(precision_score(y_test, y_pred, average='micro')))
    #print('Recall: {:.4f}'.format(recall_score(y_test, y_pred, average='micro')))
    #print('Accuracy: {:.4f}'.format(accuracy_score(y_test, y_pred)))
    predict = clf.predict(X_test)
    auc_roc = roc_auc_score(y_test, y_pred[:,1])
    print('Our MLP classifier obtains an AUC-ROC of {:.4f}.'.format(auc_roc))
    #print('F-measure: {:.4f}'.format(f1_score(y_test, y_pred.round(), average='micro')))
    print('The accuracy of classifying is {:.3f} %'.format(clf.score(X_test, y_test)*100))
    print('Confusion Matrix:')
    #print(confusion_matrix(y_test,y_pred))
    print(confusion_matrix(y_test,predict))
    print('Report:')
    #print(classification_report(y_test,y_pred))
    print(classification_report(y_test,predict))
    print("\n")
checkClf('KNN',KNeighborsClassifier(n_neighbors=91, algorithm='auto',weights='uniform'))
checkClf('SVC' , svm.SVC(kernel='linear', probability=True))
checkClf('RFC' , RandomForestClassifier(n_estimators=200))
checkClf('MLP' , MLPClassifier(max_iter = 4000,solver='sgd'))

Model name: KNN
Our MLP classifier obtains an AUC-ROC of 0.8229.
The accuracy of classifying is 80.568 %
Confusion Matrix:
[[ 3413  4889]
 [  417 18586]]
Report:
              precision    recall  f1-score   support

           0       0.89      0.41      0.56      8302
           1       0.79      0.98      0.88     19003

    accuracy                           0.81     27305
   macro avg       0.84      0.69      0.72     27305
weighted avg       0.82      0.81      0.78     27305



Model name: SVC
Our MLP classifier obtains an AUC-ROC of 0.6655.
The accuracy of classifying is 81.542 %
Confusion Matrix:
[[ 3728  4574]
 [  466 18537]]
Report:
              precision    recall  f1-score   support

           0       0.89      0.45      0.60      8302
           1       0.80      0.98      0.88     19003

    accuracy                           0.82     27305
   macro avg       0.85      0.71      0.74     27305
weighted avg       0.83      0.82      0.79     27305



Model name: RFC
Ou

In [ ]:
The SVC model has, slightly, the best accuracy of the tested models. However, its AUC-ROC is the worst among them.

In [30]:
def plot_learning_curve(estimator, title, X, y, axes=None, ylim=None, cv=None,
                        n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5)):
    if axes is None:
        _, axes = plt.subplots(1, 3, figsize=(20, 5))

    axes[0].set_title(title)
    if ylim is not None:
        axes[0].set_ylim(*ylim)
    axes[0].set_xlabel("Training examples")
    axes[0].set_ylabel("Score")

    train_sizes, train_scores, test_scores, fit_times, _ = \
        learning_curve(estimator, X, y, cv=cv, n_jobs=n_jobs,
                       train_sizes=train_sizes,
                       return_times=True)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    fit_times_mean = np.mean(fit_times, axis=1)
    fit_times_std = np.std(fit_times, axis=1)

    # Plot learning curve
    axes[0].grid()
    axes[0].fill_between(train_sizes, train_scores_mean - train_scores_std,
                         train_scores_mean + train_scores_std, alpha=0.1,
                         color="r")
    axes[0].fill_between(train_sizes, test_scores_mean - test_scores_std,
                         test_scores_mean + test_scores_std, alpha=0.1,
                         color="g")
    axes[0].plot(train_sizes, train_scores_mean, 'o-', color="r",
                 label="Training score")
    axes[0].plot(train_sizes, test_scores_mean, 'o-', color="g",
                 label="Cross-validation score")
    axes[0].legend(loc="best")

    # Plot n_samples vs fit_times
    axes[1].grid()
    axes[1].plot(train_sizes, fit_times_mean, 'o-')
    axes[1].fill_between(train_sizes, fit_times_mean - fit_times_std,
                         fit_times_mean + fit_times_std, alpha=0.1)
    axes[1].set_xlabel("Training examples")
    axes[1].set_ylabel("fit_times")
    axes[1].set_title("Scalability of the model")

    # Plot fit_time vs score
    axes[2].grid()
    axes[2].plot(fit_times_mean, test_scores_mean, 'o-')
    axes[2].fill_between(fit_times_mean, test_scores_mean - test_scores_std,
                         test_scores_mean + test_scores_std, alpha=0.1)
    axes[2].set_xlabel("fit_times")
    axes[2].set_ylabel("Score")
    axes[2].set_title("Performance of the model")

    return plt

In [ ]:
fig, axes = plt.subplots(3, 2, figsize=(10, 15))


title = "Learning Curves (SVM, linear kernel))"
cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)

estimator = svm.SVC(kernel='linear', probability=True)
plot_learning_curve(estimator, title, X, y, axes=axes[:, 0], ylim=(0.5, 1.01),
                    cv=cv, n_jobs=4)

title = r"Learning Curves (RFC, 200 estimators)"
cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=0)
estimator = RandomForestClassifier(n_estimators=200)
plot_learning_curve(estimator, title, X, y, axes=axes[:, 1], ylim=(0.5, 1.01),
                    cv=cv, n_jobs=4)

plt.show()